In [1]:
import numpy as np
import csv
import os
import time
import json
import mysql.connector as mysql

In [2]:
#  pure mysql python connector approach to the mysql connection
#   offers easy control over the BLOB type embeddings
client_config = {'unix_socket':'/home/brendanchambers/.sql.sock',
                            'database':'test_pubmed',
                            'use_pure':True}  # pure python mode is important for reading blobs of bytes
write_size = 10000  # write to db every 10K rows
table_name = 'scibert_mean_embedding'  # created in mysql terminal client
# '''CREATE TABLE {}             # <- like this
#                (pmid int NOT NULL,
#                embedding BLOB NOT NULL,
#                PRIMARY KEY (pmid))'''.format(table_name)

path2embeddings = '/project2/jevans/brendan/pubmed_data_processing/scibert_embedding_chunks/'


## helper function

In [3]:
def write_data_to_db(entries, table_name):
    db = mysql.connect(**client_config)
    sql = '''INSERT INTO {} (pmid,embedding)
             VALUES (%s, %s)
             ON DUPLICATE KEY UPDATE
             pmid=values(pmid), embedding=values(embedding)'''.format(table_name)
    cursor = db.cursor()
    cursor.executemany(sql, entries)
    cursor.close()
    db.commit()

## read csv output files and write to db

In [4]:
start_time = time.time()

for chunk_filename in os.listdir(path2embeddings):
    data = []
    print('processing {}...'.format(chunk_filename))
    
    with open(path2embeddings + chunk_filename) as f:
        csvreader = csv.reader(f, delimiter=' ')
        # note: there is no header in these chunk files

        for idx, row in enumerate(csvreader):
            pmid = int(row[0])
            
            embedding_blob = np.array(json.loads(row[1])).tobytes(
                                                        order='C')
            data.append((pmid, embedding_blob))
            
            if len(data) > write_size:  # write to db intermittently
                write_data_to_db(data, table_name)
                data = []           
                # reset data
                
    write_data_to_db(data, table_name)  # empty the buffer at the end

end_time = time.time()
print('elapsed: {}'.format(end_time - start_time))


processing chunk_40.csv...
processing chunk_8.csv...
processing chunk_49.csv...
processing chunk_41.csv...
processing chunk_27.csv...
processing chunk_36.csv...
processing chunk_46.csv...
processing chunk_23.csv...
processing chunk_13.csv...
processing chunk_19.csv...
processing chunk_38.csv...
processing chunk_10.csv...
processing chunk_30.csv...
processing chunk_11.csv...
processing chunk_2.csv...
processing chunk_28.csv...
processing chunk_24.csv...
processing chunk_45.csv...
processing chunk_18.csv...
processing chunk_9.csv...
processing chunk_5.csv...
processing chunk_37.csv...
processing chunk_12.csv...
processing chunk_32.csv...
processing chunk_33.csv...
processing chunk_47.csv...
processing chunk_1.csv...
processing chunk_42.csv...
processing chunk_20.csv...
processing chunk_16.csv...
processing chunk_15.csv...
processing chunk_21.csv...
processing chunk_14.csv...
processing chunk_17.csv...
processing chunk_7.csv...
processing chunk_31.csv...
processing chunk_25.csv...
process